In [ ]:
!curl --request GET --url host.docker.internal:21434/api/tags

In [ ]:
from langchain_ollama import ChatOllama

### llm 
local_llm="deepseek-r1:1.5b-qwen-distill-fp16"
llm = ChatOllama(model=local_llm, temperature=0, base_url="http://host.docker.internal:21434")
llm_json = ChatOllama(model=local_llm, temperature=0, return_json=True, base_url="http://host.docker.internal:21434")

In [ ]:
msg = llm.invoke("What is the capital of France?")
msg.content

__________________

# GROQ test

In [ ]:
import os

from groq import Groq
from dotenv import load_dotenv
load_dotenv("../../groq.env")


In [ ]:
client = Groq(
    # This is the default and can be omitted
    api_key=os.getenv("GROQ_API_KEY"),
)

In [ ]:
model_name = os.getenv("GROQ_LLM")
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "you are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model=model_name,
    reasoning_format={"type": "groq"},
    temperature=0.6,
    max_completion_tokens=131072,
    top_p=0.95,
    stream=True,
    stop=None,
    
)

In [ ]:

for chunk in chat_completion:
    print(chunk.choices[0].delta.content or "", end="")

__________


# GROQ langchain test

In [9]:
import os
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from ollama_deep_researcher.prompts import (
    query_writer_instructions,
    summarizer_instructions,
    reflection_instructions,
    get_current_date,
)
from dotenv import load_dotenv
load_dotenv("../../groq.env")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
model_name = os.getenv("GROQ_LLM")

In [10]:
current_date = get_current_date()
formatted_prompt = query_writer_instructions.format(
        current_date=current_date, research_topic="fast language models"
    )

In [11]:
llm = ChatGroq(
    model=model_name,
    temperature=0,
    response_format={"type": "json_object"},
)
#llm_json_mode = llm.with_structured_output(method="json_mode")

/home/jens/.cache/pypoetry/virtualenvs/ollama-deep-researcher-AmCmqQEy-py3.12/lib/python3.12/site-packages/langchain_groq/chat_models.py:364: UserWarning: WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.
  warnings.warn(


In [ ]:
result = llm_json_mode.invoke(
    [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content=f"Generate a query for web search:"),
    ]
)

In [12]:
result2 = llm.invoke(
    [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content=f"Generate a query for web search:"),
    ]
)

In [13]:
print(result2.content)

{    "query": "recent advancements in optimizing language models for speed",
    "rationale": "This query focuses on the latest developments in making language models faster, which is crucial for real-time applications and efficient processing."
}
